In [1]:
# Importing libraries
import math
import numpy as np
import pandas as pd

from scipy.interpolate import make_interp_spline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
# List of optimum diameters
D = [4,5,6,8,10,12,14,16,18,20,24]
# List of (inner diameter, optimum diameter)
Di = [(4.026,4),(5.047,5),(6.065,6),(7.981,8),(10.02,10),(11.938,12),(13.125,14),(15,16),(16.874,18),(18.814,20),(22.626,24)]
# List of flow rate values
Q = [100,120,130,140,160,180,200,210,220,250]
# List of density values
Density = [700,710,720,730,750,760,780,800,820,840]
# List of viscosity values
Viscosity = [10,15,20,25,30,35,40,45,50,55]

In [3]:
# Defining Functions

def calculate_fixed_cost(d_optimum):
    return 0.335*(0.05+0.015)*(6.5+1)*6.5*(d_optimum**1.5)

def calculate_friction_factor(d_optimum, d_inner, flow_rate, density, viscosity):
    r = 0.0018/d_optimum
    re = 13.924*(flow_rate*density/(viscosity*d_inner))
    x = math.log(1/((7/re)**0.9+(0.27*r)), math.e)
    a = (2.457*x)**16
    b = (37.53/re)**16
    return (8*((8/re)**12+(1/(a+b)**1.5))**(1/12))

def calculate_total_losses(f, flow_rate, d_inner):
    hf = 603.042*(f*(flow_rate**2)*(0.0003048))/(d_inner**5)
    return (1.15*hf)

def calculate_operating_cost(flow_rate, density, total_losses):
    return (flow_rate*density*total_losses*8000*0.073)/(361930.3*0.65)

def fit_optimum_diameter_with_cost(optimum_diameter_list, total_cost_list):
    optimum_diameters = np.linspace(4 , 24, 1001)
    spline = make_interp_spline(optimum_diameter_list, total_cost_list, k=3)
    total_cost_values = spline(optimum_diameters)
    return optimum_diameters, total_cost_values

def find_optimum_diameter(optimum_diameters, total_cost_values):
    minimum_cost_index = np.argmin(total_cost_values)
    return optimum_diameters[minimum_cost_index]

In [4]:
# Define Dataset
data = pd.DataFrame({"flow_rate": [], "density": [], "viscosity": [], "optimum_diameter": []})
data

,flow_rate,density,viscosity,optimum_diameter


In [5]:
# Loop through Flow rate values
for q in Q:
    # Loop through Density values
    for d in Density:
        # Loop through Viscosity values
        for mu in Viscosity:
            optimum_diameter_list = []     # To store diameter values
            total_cost_list = []           # to store total cost for each diameter
            # Loop through diameters
            for di, do in Di:
                # Calculate fixed cost
                fixed_cost = calculate_fixed_cost(d_optimum=do)
                
                # Calculate friction factor
                f = calculate_friction_factor(d_optimum=do, d_inner=di, flow_rate=q, density=d, viscosity=mu)
                
                # Calculate total losses
                ht = calculate_total_losses(f=f, flow_rate=q, d_inner=di)
                
                # Calculate operating cost
                operating_cost = calculate_operating_cost(flow_rate=q, density=d, total_losses=ht)
                
                # Calculate total cost
                total_cost = operating_cost + fixed_cost
                
                # Add the values of optimum diameter and the corresponding total cost to lists
                optimum_diameter_list.append(do)
                total_cost_list.append(total_cost)
            
            # get optimum diameter accurately
            optimum_diameters, total_cost_values = fit_optimum_diameter_with_cost(optimum_diameter_list, total_cost_list)
            optimum_diameter = find_optimum_diameter(optimum_diameters, total_cost_values)
            
            # Add new row to dataset
            new_row = pd.DataFrame({"flow_rate": [q], "density": [d], "viscosity": [mu], "optimum_diameter": [optimum_diameter]})
            data = pd.concat([data, new_row], axis=0)

In [6]:
# Show dataset
data.reset_index(inplace=True)
data.drop(columns="index", inplace=True)
data

,flow_rate,density,viscosity,optimum_diameter
0,100.0,700.0,10.0,4.94
1,100.0,700.0,15.0,5.00
2,100.0,700.0,20.0,5.06
3,100.0,700.0,25.0,5.10
4,100.0,700.0,30.0,5.12
...,...,...,...,...
995,250.0,840.0,35.0,7.74
996,250.0,840.0,40.0,7.78
997,250.0,840.0,45.0,7.82
998,250.0,840.0,50.0,7.86


In [7]:
# make logarithmic distribution to data and separate dataset to (X_train, X_test, y_train, y_test)
X = data.drop(columns="optimum_diameter")
X = np.log10(X)
y = data["optimum_diameter"]
y = np.log10(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Train and test and evaluate model
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("Train r2_score =", r2_score(y_train_pred, y_train))
print("Train MAE_score =", mean_absolute_error(y_train_pred, y_train))
print("Test r2_score =", r2_score(y_test_pred, y_test))
print("Test MAE_score =", mean_absolute_error(y_test_pred, y_test))

Train r2_score = 0.9938946955772389
Train MAE_score = 0.003268327135572568
Test r2_score = 0.994187661191643
Test MAE_score = 0.003325161735998188


In [9]:
# get model intercept and coeffecients
intercept = model.intercept_
coeffecients = model.coef_
print(intercept)
print(coeffecients)

-0.5570345157029849
[0.44143514 0.11268439 0.03724025]


In [10]:
# Get imperical equation for optimum diameter
constant = np.round(10**intercept, 3)
a, b, c = np.round(np.array(coeffecients), 3)
print(f"D_optimum = {constant} (Q)^{a} * (ru)^{b} * (mu)^{c}")

D_optimum = 0.277 (Q)^0.441 * (ru)^0.113 * (mu)^0.037


In [11]:
def make_prediction(q, ru, mu):
    return 0.277 *(q**0.441)*(ru**0.113)*(mu**0.037)

In [12]:
q = 250
ru = 800
mu = 35
print("Optimum Diameter =", make_prediction(q, ru, mu), "in")

Optimum Diameter = 7.676283382721798 in
